In [ ]:
!pip install transformers datasets sentencepiece torch accelerate

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Task
Train a T5 model for decomposition tasks using the dataset from "https://raw.githubusercontent.com/openai/grade-school-math/refs/heads/master/grade_school_math/data/train.jsonl".

## Load the dataset

### Subtask:
Load the dataset from the provided URL.


**Reasoning**:
The subtask requires loading a dataset from a URL in JSON format using the `datasets` library. This can be achieved by importing `load_dataset` and calling it with the specified URL and format.



In [ ]:
from datasets import load_dataset

dataset = load_dataset('json', data_files='https://raw.githubusercontent.com/openai/grade-school-math/refs/heads/master/grade_school_math/data/train.jsonl')

Generating train split: 0 examples [00:00, ? examples/s]

## Preprocess the data

### Subtask:
Tokenize the input questions and the corresponding decomposition tasks.


**Reasoning**:
Define and apply the preprocessing function to tokenize the dataset.



In [ ]:
def preprocess_function(examples):
    inputs = [f"decompose: {q}" for q in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["question", "answer"])
display(tokenized_datasets)

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7473
    })
})

## Define training arguments

### Subtask:
Set up the training parameters such as learning rate, batch size, and number of epochs.


**Reasoning**:
Import the `TrainingArguments` class and instantiate it with the required parameters for training.



In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch"
)

**Reasoning**:
The previous command failed because `evaluation_strategy` is not a valid argument for `TrainingArguments` in this version of the library. I need to remove this argument and try again. I will also remove `save_strategy` as it might also be an invalid argument.



In [ ]:
from transformers import TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=100,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
)

## Train the model

### Subtask:
Train the T5 model on the preprocessed dataset using the defined training arguments.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipython-input-2495732291.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kshitijbudholiya2006 (kshitijbudholiya2006-institute-of-technology-management-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.580700
200,0.468100
300,0.415500
400,0.392400
500,0.395600
600,0.377400
700,0.374600
800,0.353300
900,0.355800
1000,0.358400


TrainOutput(global_step=9345, training_loss=0.29151669028227567, metrics={'train_runtime': 4044.8217, 'train_samples_per_second': 9.238, 'train_steps_per_second': 2.31, 'total_flos': 5057046412001280.0, 'train_loss': 0.29151669028227567, 'epoch': 5.0})